In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from datasets import load_dataset
from tqdm import tqdm

from qasper.dataset_reader import QasperReader
from qasper.models import qasper, gpt35
from qasper.utils import print_wrap

/Users/ag2435/anaconda3/envs/arxiv-agent/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
dataset = load_dataset("allenai/qasper")

/Users/ag2435/anaconda3/envs/arxiv-agent/lib/python3.12/site-packages/datasets/load.py:1461: FutureWarning: The repository for allenai/qasper contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/allenai/qasper
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(
Generating test split: 100%|██████████| 416/416 [00:00<00:00, 1751.79 examples/s]


In [4]:
dataset['validation']

Dataset({
    features: ['id', 'title', 'abstract', 'full_text', 'qas', 'figures_and_tables'],
    num_rows: 281
})

In [5]:
dataset['validation'][0]

{'id': '1912.01214',
 'title': 'Cross-lingual Pre-training Based Transfer for Zero-shot Neural Machine Translation',
 'abstract': 'Transfer learning between different language pairs has shown its effectiveness for Neural Machine Translation (NMT) in low-resource scenario. However, existing transfer methods involving a common target language are far from success in the extreme scenario of zero-shot translation, due to the language space mismatch problem between transferor (the parent model) and transferee (the child model) on the source side. To address this challenge, we propose an effective transfer learning approach based on cross-lingual pre-training. Our key idea is to make all source languages share the same feature space and thus enable a smooth transition for zero-shot translation. To this end, we introduce one monolingual pre-training method and two bilingual pre-training methods to obtain a universal encoder for different languages. Once the universal encoder is constructed, t

In [6]:
reader = QasperReader()

In [7]:
def instance_generator(split):
    for article in split:
        for instance in reader._article_to_instances(article):
            yield instance

In [8]:
# randomly sample 100 instances
import random
random.seed(42)
instances = list(instance_generator(dataset['validation']))
instances = random.sample(instances, 100)

In [9]:
reader._stats

defaultdict(int,
            {'number of documents': 281,
             'number of questions': 1005,
             'number of answers': 3015,
             'questions with multiple answers': 1005,
             'extractive questions': 962,
             'extractive questions with multiple spans': 406,
             'multiple_evidence_spans_count': 536,
             'answers with table or figure as evidence': 212,
             'freeform answers': 431,
             'yes/no questions': 208,
             'answers with no evidence': 212,
             'unanswerable questions': 163,
             'number of truncated contexts': 15})

In [10]:
len(instances)

100

In [11]:
instance = instances[0]
print(instance.keys())
print('QUESTION WITH CONTEXT:')
# print_wrap(instance['s_question_with_context'])
print(instance['s_question_with_context'])

dict_keys(['question_with_context', 's_question_with_context', 'paragraph_indices', 'global_attention_mask', 'evidence', 'answer', 'metadata'])
QUESTION WITH CONTEXT:
Did they experiment with this new dataset?
Introduction
How humans process language has become increasingly relevant in natural language processing since physiological data during language understanding is more accessible and recorded with less effort. In this work, we focus on eye-tracking and electroencephalography (EEG) recordings to capture the reading process. On one hand, eye movement data provides millisecond-accurate records about where humans look when they are reading, and is highly correlated with the cognitive load associated with different stages of text processing. On the other hand, EEG records electrical brain activity across the scalp and is a direct measure of physiological processes, including language processing. The combination of both measurement methods enables us to study the language understanding

In [12]:
print_wrap(' '.join(instance['question_with_context']))

<s> Did Ġthey Ġexperiment Ġwith Ġthis Ġnew Ġdataset ? </s> Introduction </s> How
Ġhumans Ġprocess Ġlanguage Ġhas Ġbecome Ġincreasingly Ġrelevant Ġin Ġnatural
Ġlanguage Ġprocessing Ġsince Ġphysiological Ġdata Ġduring Ġlanguage
Ġunderstanding Ġis Ġmore Ġaccessible Ġand Ġrecorded Ġwith Ġless Ġeffort . ĠIn
Ġthis Ġwork , Ġwe Ġfocus Ġon Ġeye - tracking Ġand Ġelectro ence phal ography Ġ(
EE G ) Ġrecordings Ġto Ġcapture Ġthe Ġreading Ġprocess . ĠOn Ġone Ġhand , Ġeye
Ġmovement Ġdata Ġprovides Ġmillisec ond - acc urate Ġrecords Ġabout Ġwhere
Ġhumans Ġlook Ġwhen Ġthey Ġare Ġreading , Ġand Ġis Ġhighly Ġcorrelated Ġwith
Ġthe Ġcognitive Ġload Ġassociated Ġwith Ġdifferent Ġstages Ġof Ġtext Ġprocessing
. ĠOn Ġthe Ġother Ġhand , ĠEEG Ġrecords Ġelectrical Ġbrain Ġactivity Ġacross
Ġthe Ġscalp Ġand Ġis Ġa Ġdirect Ġmeasure Ġof Ġphysiological Ġprocesses ,
Ġincluding Ġlanguage Ġprocessing . ĠThe Ġcombination Ġof Ġboth Ġmeasurement
Ġmethods Ġenables Ġus Ġto Ġstudy Ġthe Ġlanguage Ġunderstanding Ġprocess Ġin Ġa

In [13]:
print('QUESTION:', instance['metadata']['question'])
print('ANSWER:', instance['answer'])

QUESTION: Did they experiment with this new dataset?
ANSWER: No


In [14]:
instance['metadata']['article_id']

'1912.00903'

## Evaluate F1 score

In [15]:
from qasper.evaluator import token_f1_score, get_answers_and_evidence, evaluate

In [16]:
qasper_answer = qasper.predict(instance)[0]
print(qasper_answer)

Input ids are automatically padded from 4697 to 5120 to be a multiple of `config.attention_window`: 1024
/Users/ag2435/anaconda3/envs/arxiv-agent/lib/python3.12/site-packages/transformers/generation/utils.py:1132: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


, namely one word, i.e. we are examining the effects of attention delay on


In [17]:
token_f1_score(qasper_answer, instance['answer'])

0

In [18]:
gpt35_answer = gpt35.predict(instance)
print_wrap(gpt35_answer)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Yes, the researchers experimented with the new dataset, Zurich Cognitive
Language Processing Corpus (ZuCo) 2.0. This corpus includes simultaneous eye
movement and brain activity recordings during natural reading and task-specific
reading during annotation. The dataset consists of physiological data from 18
subjects reading 739 English sentences from Wikipedia. The researchers conducted
a detailed technical validation of the data to ensure the quality of the
recordings.


The corpus construction involved recording data from 19 participants, with data
from one participant discarded due to technical issues. The participants read
sentences selected from the Wikipedia corpus, and the dataset included both
normal reading and task-specific reading paradigms. The experimental design
included participants reading sentences at their own speed, using a control pad
to move to the next sentence and answer control questions.


Data acquisition took place in a controlled environment, with eye-trackin

In [19]:
token_f1_score(gpt35_answer, instance['answer'])

0

In [20]:
def evaluate(gold, predicted):
    max_answer_f1s = []
    max_evidence_f1s = []
    max_answer_f1s_by_type = {
        "extractive": [],
        "abstractive": [],
        "boolean": [],
        "none": [],
    }
    num_missing_predictions = 0
    for question_id in gold:
        if question_id not in predicted:
            num_missing_predictions += 1
            max_answer_f1s.append(0.0)
            max_evidence_f1s.append(0.0)
            continue
        answer_f1s_and_types = [
            (token_f1_score(predicted[question_id]["answer"], reference["answer"]),
             reference["type"])
            for reference in gold[question_id]
        ]
        max_answer_f1, answer_type = sorted(answer_f1s_and_types, key=lambda x: x[0], reverse=True)[0]
        max_answer_f1s.append(max_answer_f1)
        max_answer_f1s_by_type[answer_type].append(max_answer_f1)
        # evidence_f1s = [
        #     paragraph_f1_score(predicted[question_id]["evidence"], reference["evidence"])
        #     for reference in gold[question_id]
        # ]
        # max_evidence_f1s.append(max(evidence_f1s))

    mean = lambda x: sum(x) / len(x) if x else 0.0
    return {
        "Answer F1": mean(max_answer_f1s),
        "Answer F1 by type": {key: mean(value) for key, value in max_answer_f1s_by_type.items()},
        # "Evidence F1": mean(max_evidence_f1s),
        "Missing predictions": num_missing_predictions
    }

In [21]:
# gold_data = json.load(open(args.gold))
gold_answers_and_evidence = get_answers_and_evidence(dataset['validation'])

In [22]:
gold_answers_and_evidence.keys()

dict_keys(['...'])

In [23]:
gold_answers_and_evidence['b6f15fb6279b82e34a5bf4828b7b5ddabfdf1d54']

[{'answer': 'BIBREF19, BIBREF20',
  'evidence': ['Table TABREF19 and TABREF26 report zero-shot results on Europarl and Multi-UN evaluation sets, respectively. We compare our approaches with related approaches of pivoting, multilingual NMT (MNMT) BIBREF19, and cross-lingual transfer without pretraining BIBREF16. The results show that our approaches consistently outperform other approaches across languages and datasets, especially surpass pivoting, which is a strong baseline in the zero-shot scenario that multilingual NMT systems often fail to beat BIBREF19, BIBREF20, BIBREF23. Pivoting translates source to pivot then to target in two steps, causing inefficient translation process. Our approaches use one encoder-decoder model to translate between any zero-shot directions, which is more efficient than pivoting. Regarding the comparison between transfer approaches, our cross-lingual pretraining based transfer outperforms transfer method that does not use pretraining by a large margin.'],
 

In [ ]:
predicted_answers_and_evidence = {}

In [25]:
for instance in tqdm(instances):
    question_id = instance["metadata"]["question_id"]

    if question_id in predicted_answers_and_evidence: # keep this to conserve API requests
        continue

    # prediction_data = json.loads(line)
    # pred_answer = qasper.predict(instance)[0]
    pred_answer = gpt35.predict(instance)

    predicted_answers_and_evidence[question_id] = {
        "answer": pred_answer,
        # "evidence": prediction_data["predicted_evidence"]
    }


100%|██████████| 100/100 [07:07<00:00,  4.27s/it]


In [ ]:
# save predictions
import json
with open('output/gpt35-predictions.json', 'w') as f:
    json.dump(predicted_answers_and_evidence, f)

In [28]:
evaluation_output = evaluate(
    {k:v for k, v in gold_answers_and_evidence.items() \
        if k in predicted_answers_and_evidence}, 
    predicted_answers_and_evidence)

In [29]:
len(gold_answers_and_evidence)

1005

In [30]:
evaluation_output

{'Answer F1': 0.06489175253739486,
 'Answer F1 by type': {'extractive': 0.08545044887734807,
  'abstractive': 0.08790371215230747,
  'boolean': 0.0013445793337097684,
  'none': 0.0},
 'Missing predictions': 0}

In [ ]:
# print(json.dumps(evaluation_output, indent=2))